In [23]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

In [2]:
# !head -n 5 ../data/results-gpt4o-mini-cosine.csv
df = pd.read_csv('../data/results-gpt4o-mini-cosine.csv', nrows=300)

## Q1. Getting the embeddings model

In [9]:
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
answer_llm = df.iloc[0].answer_llm
vector = embedding_model.encode(answer_llm)
print("What's the first value of the resulting vector?: ", round(vector[0], 2))

What's the first value of the resulting vector?:  -0.42


## Q2. Computing the dot product

In [19]:
# Function to compute embeddings
def emb_cal(row, column):
    value = row[column]
    return embedding_model.encode(value)

# Compute embeddings for both columns
df['answer_llm_vector'] = df.apply(lambda row: emb_cal(row, column='answer_llm'), axis=1)
df['answer_orig_vector'] = df.apply(lambda row: emb_cal(row, column='answer_orig'), axis=1)

In [27]:
# Compute the dot product and store in 'score'
df['score'] = df.apply(lambda row: np.dot(row['answer_llm_vector'], row['answer_orig_vector']), axis=1)

print("What's the 75% percentile of the score?: ", round(df['score'].quantile(0.75), 2))

What's the 75% percentile of the score?:  31.67
